In [1]:
import sys
!{sys.executable} -m pip install pandas requests lxml html5lib

  Using cached https://files.pythonhosted.org/packages/e1/d8/feeb346d41f181e83fba45224ab14a8d8af019b48af742e047f3845d8cff/pandas-0.23.4-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/5d/d4/e81be10be160a6323cf5f29f1eabc9693080cb16780a2e19c96091ee37ee/lxml-4.3.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/a5/62/bbd2be0e7943ec8504b517e62bab011b4946e1258842bc159e5dfde15b96/html5lib-1.0.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ff/7f/9d804d2348471c67a7d8b5f84f9bc59fd1cefa148986f2b74552f8573555/numpy-1.15.4-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/61/28/1d3920e4d1d50b19bc5d24398a7cd85cc7b9a75a490570d5a30c57622d34/pytz-2018.9-py2.py3-none-any.whl


Import packages

In [2]:
import requests
import re
import numpy as np
import pandas as pd
import html5lib

1. We should collect all neighbourhoods of Moscow, Russia. At this tep we can get all postal codesfrom http://mosopen.ru/streets/post_codes_list. But we must pre-process data to have unique postal codes of regions.

Download **Moscow** postal codes data from site

In [ ]:
moscow_postcodes_link='http://mosopen.ru/streets/post_codes_list'
raw_moscow_postcodes_page=requests.get(moscow_postcodes_link)
moscow_postcodes_page=raw_moscow_postcodes_page.text

Find and parse table from page

In [3]:
page_table_start = moscow_postcodes_page.find('<table class="table_list">')
page_table_end = moscow_postcodes_page.find('</table>', page_table_start)
print(page_table_start, page_table_end)

17716 66404


Read HTML table to pandas DataFrame

In [4]:
moscow_postcodes_page_table = moscow_postcodes_page[page_table_start : page_table_end]

In [5]:
moscow_postcodes_df = pd.read_html(moscow_postcodes_page_table, header =0)[0]
# Rename columns
moscow_postcodes_df.columns = ['city_code', 'postal_code']
# Transform city_code to string
moscow_postcodes_df['city_code']= moscow_postcodes_df['city_code'].astype(str)

In [6]:
postcodes = moscow_postcodes_df['postal_code'].str.split(',', expand=True).stack().str.strip().reset_index(level=1, drop=True)
postcodes = postcodes.rename('postal_code')

In [7]:
moscow_postcodes_df = moscow_postcodes_df.drop(['postal_code'], axis=1)

In [8]:
moscow_postcodes_df = pd.concat([moscow_postcodes_df, postcodes], axis=1)

In [9]:
moscow_postcodes_df['postal_code'] = moscow_postcodes_df['city_code'] + moscow_postcodes_df['postal_code']

In [10]:
moscow_postcodes_df = moscow_postcodes_df.drop(['city_code'], axis=1)

In [12]:
moscow_postcodes_df

,postal_code
0,101000
1,103070
1,103132
1,103274
2,105005
2,105037
2,105043
2,105062
2,105064
2,105066
